In [1]:
!pip install elasticsearch

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import json
from elasticsearch.helpers import bulk
import pandas as pd

# Corpus Path 
corpus_path = r'..\trec-covid\corpus.jsonl'

# reading
data = []
with open(corpus_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

# Dataframe creation
# This is out corpus
df = pd.DataFrame(data)

In [3]:
!curl http://localhost:9200/

{
  "name" : "DESKTOP-7VCER44",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "4KOjzn0zQLCGNiwWwmPhMg",
  "version" : {
    "number" : "9.0.1",
    "build_flavor" : "default",
    "build_type" : "zip",
    "build_hash" : "73f7594ea00db50aa7e941e151a5b3985f01e364",
    "build_date" : "2025-04-30T10:07:41.393025990Z",
    "build_snapshot" : false,
    "lucene_version" : "10.1.0",
    "minimum_wire_compatibility_version" : "8.18.0",
    "minimum_index_compatibility_version" : "8.0.0"
  },
  "tagline" : "You Know, for Search"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   539  100   539    0     0   4241      0 --:--:-- --:--:-- --:--:--  4244
100   539  100   539    0     0   4230      0 --:--:-- --:--:-- --:--:--  4210


In [4]:
import re

# The code cleans and standardizes text by converting it to lowercase, 
# removing HTML tags, digits, and punctuation, and collapsing multiple spaces into one.
def clear_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"<[^>]+>", " ", text)
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"[^\w\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    return text

df["clean_text"] = (df["title"].fillna("") + " " + df["text"].fillna("")).apply(clear_text)
# Remove rows where clean_text is empty
df = df[df["clean_text"] != ""]

In [5]:
print(df[["_id", "title","clean_text","text"]].head())

        _id                                              title  \
0  ug7v899j  Clinical features of culture-proven Mycoplasma...   
1  02tnwd4m  Nitric oxide: a pro-inflammatory mediator in l...   
2  ejv2xln0    Surfactant protein-D and pulmonary host defense   
3  2b73a28n               Role of endothelin-1 in lung disease   
4  9785vg6d  Gene expression in epithelial cells in respons...   

                                          clean_text  \
0  clinical features of culture proven mycoplasma...   
1  nitric oxide a pro inflammatory mediator in lu...   
2  surfactant protein d and pulmonary host defens...   
3  role of endothelin in lung disease endothelin ...   
4  gene expression in epithelial cells in respons...   

                                                text  
0  OBJECTIVE: This retrospective chart review des...  
1  Inflammatory diseases of the respiratory tract...  
2  Surfactant protein-D (SP-D) participates in th...  
3  Endothelin-1 (ET-1) is a 21 amino acid pept

In [6]:
print(df.columns)

Index(['_id', 'title', 'text', 'metadata', 'clean_text'], dtype='object')


In [7]:
from elasticsearch import Elasticsearch, helpers

client = Elasticsearch("http://localhost:9200")

index_name = "my_index"

vsm_settings={
    "settings": {
        "number_of_shards": 1,
        "similarity": {
            "scripted_tfidf": {
                "type": "scripted",
                "script": {
                    "source": "double tf = Math.sqrt(doc.freq); double idf = Math.log((field.docCount+1.0)/(term.docFreq+1.0)) + 1.0; double norm = 1/Math.sqrt(doc.length); return query.boost * tf * idf * norm;"
                }
            }
        },        
        "analysis": {
            "analyzer": {
                "default": {
                    "type": "english"
                },
                "default_search": {
                    "type": "english"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "text": {
                "type": "text",
                "similarity": "scripted_tfidf"
        }
    }
}
}



In [8]:
client.indices.delete(index="my_index")

ObjectApiResponse({'acknowledged': True})

In [9]:
# Create the index with mappings and settings
if not client.indices.exists(index=index_name):
    response = client.indices.create(index=index_name, body=vsm_settings)
    print("Index created:", response)
else:
    print(f"Index '{index_name}' already exists.")


docs = [
    {
        "_id": row["_id"],
        "_source": {
            "text": row["clean_text"]
        }
    }
    for _, row in df.iterrows()
]
bulk_response = helpers.bulk(client, docs,index=index_name)
print("Success:", bulk_response)

Index created: {'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'}
Success: (171331, [])


In [10]:
import json

# take queries from file 
queries = []
with open(r"..\trec-covid\queries.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        queries.append(json.loads(line))

In [17]:
import os

def pretty_search_response(response, file_path, query_id, run_id="my_run"):
    with open(file_path, "w", encoding="utf-8") as f:
        if len(response["hits"]["hits"]) == 0:
            msg = "Your search returned no results for this query."
            print(msg)
            f.write(f"{query_id} Q0 no_results 1 0.0 {run_id}\n") # Output even if no results
        else:
            rank = 1
            for hit in response["hits"]["hits"]:
                doc_id = hit["_id"]
                score = hit["_score"]
                f.write(f"{query_id} Q0 {doc_id} {rank} {score} {run_id}\n")
                rank += 1


In [18]:
for q in queries:
    query_text = q["metadata"]["query"]
    query_id = q["_id"]

    response = client.search(
        index=index_name,
        size=50,
        query={
            "match": {
                "text": query_text
            }
        }
    )
    base_path = r"res\50new"
    file_name = os.path.join(base_path, f"query_{query_id}.txt")

    pretty_search_response(response, file_name, query_id)